## Evaluate a RAG application

This example uses [LangChain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

Reference video: [YouTube](https://youtu.be/ZPX3W77h_1E?si=58AdU770UWDJdYo3)

In [1]:
import os
import requests
from typing import Optional
import litellm
import giskard
from dotenv import load_dotenv
import getpass
from IPython.display import Markdown

## Setup the GROQ LLM

In [2]:
# GROQ API KEY
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

if not os.getenv("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

In [3]:
giskard.llm.set_llm_model("groq/meta-llama/llama-4-scout-17b-16e-instruct")

## Setup the Ollama Embedding Model

In [5]:
giskard.llm.set_embedding_model("ollama/nomic-embed-text", api_base = "http://localhost:11434")

## Load the Chroma Vector Store

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

embeddings = OllamaEmbeddings(model="nomic-embed-text")

db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

Let's start by loading the content in a pandas DataFrame.

In [7]:
import pandas as pd
data = db.get()
documents = data.get('documents', [])
df = pd.DataFrame({'chunks': documents})
df.head()

,chunks
0,i\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...
1,ii \n \nAcknowledgments \nThe Government of Pa...
2,iii \n \nTable of Contents \n1 Executive Summa...
3,3.1 Vision ......................................
4,3.4.2 The National AI Targets ...................


We can now create a Knowledge Base using the DataFrame we created before.

In [8]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

In [10]:
import nest_asyncio
nest_asyncio.apply()

## Generate the Test Set

In [12]:
from giskard.rag import generate_testset
from giskard.rag.question_generators import simple_questions
from giskard.rag import QATestset

if os.path.exists("testset10.jsonl"):
    testset = QATestset.load("testset10.jsonl")
else:
    testset = generate_testset(
        knowledge_base,
        num_questions = 10,
        question_generators = [simple_questions],
        agent_description = "A chat assistant for the Government of Pakistan’s Ministry of Information Technology & Telecommunication (MoITT), specializing in the National Artificial Intelligence Policy."
    )
    testset.save("testset10.jsonl")

2025-08-20 17:50:06,232 pid:11484 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2025-08-20 17:51:33,546 pid:11484 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

Let's display a few samples from the test set.

In [12]:
testset.to_pandas().head(5)

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
11fde9b4-0b13-4d48-8665-113a34f7ebaa,What are the sectors that participated in the ...,"Agriculture Industry, Healthcare Industry, Man...",Document 134: 21 \n \n \n0 5 10 15 20 25\nAgri...,[],"{'question_type': 'simple', 'seed_document_id'..."
d8ba0147-d2e3-4581-aeb7-427ad95c0dc2,What are the priority social sectors for data ...,"Healthcare, Legal, Public Facilitation Service...",Document 58: I. The CoE-AI shall organize a co...,[],"{'question_type': 'simple', 'seed_document_id'..."
81922b3d-0ba2-47e9-b53f-5aee1299165b,What is one of the goals of the CoE-AI in deve...,to streamline the delivery of municipal servic...,Document 70: 6 \n \nIII. The program shall emp...,[],"{'question_type': 'simple', 'seed_document_id'..."
3db6e3ad-c384-48e3-8141-ca345b563dcb,What are some examples of chronic diseases tha...,"diabetes, hypertension, and high blood cholest...",Document 62: failure. Many Pakistanis have chr...,[],"{'question_type': 'simple', 'seed_document_id'..."
e8acc023-18b1-4753-b06f-69f0bb983344,What is the vision of the Government of Pakist...,To Embrace AI by appreciating Human Intelligen...,"Document 22: 8 \n \n3 Vision, Scope & Objectiv...",[],"{'question_type': 'simple', 'seed_document_id'..."


## Prepare the Prompt

In [13]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""You are a professional virtual assistant for the Government of Pakistan’s Ministry of Information Technology & Telecommunication (MoITT), specializing in the "National Artificial Intelligence Policy".
    Your primary role is to use the provided context to respond with accurate, concise, and helpful information about the policy’s vision, objectives, directives, targets, and related initiatives.
    You should respond to user inquiries in a professional, clear, and neutral manner, ensuring your answers are easy to understand while maintaining policy accuracy.

    If the answer is explicitly present in the retrieved context, quote or paraphrase accurately.  
    When citing, follow this style:  
    - Place citations at the end of the relevant sentence or paragraph.  
    - Use parentheses with the section name and number, e.g., *(Section 3.1 — Vision)* or *(Section 4.1.2 — Center of Excellence in AI)*.  
    - Do not use brackets like 【 】 or repeat the section name twice.  
    - If no exact section number is available, cite the nearest heading in the context.

    Only use the information provided in the context or conversation history to answer the question. **Do not fabricate or assume any details.**
    If the answer cannot be derived from the given information, politely state that the provided excerpts do not contain that information.

    Only answer the specific question asked. Do not include unrelated information or anticipate additional questions.
                                      
    Context: {context}

    Question: {question}

    Answer:""")

## Create the RAG Chain

In [14]:
# Retriever

retriever = db.as_retriever(search_kwargs={'k': 5})

In [15]:
from operator import itemgetter
from langchain.chat_models import init_chat_model

# Initialize the Response Generator LLM
model = init_chat_model("openai/gpt-oss-120b", model_provider="groq", temperature=0)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | (lambda x: x.content)
)

In [16]:
response = chain.invoke({"question": "What is the vision of Pakistan's AI Policy?"})

Markdown(response)

The vision of Pakistan’s AI Policy is to “Embrace AI by appreciating Human Intelligence and stimulating a Hybrid Intelligence ecosystem for equitable, responsible, and transparent use of AI.” *(Section 3.1 — Vision)*

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [17]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set. We'll use Giskard's built-in RAGAS metric wrappers for more reliable evaluation.

In [18]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/10 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history (Not a part of our RAG Pipeline).
* **Router**: This is a component that filters the query of the user based on his intentions (Not a part of our RAG Pipeline).
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [19]:
display(report)

Loading BokehJS ...

We can display the correctness results organized by question type.

In [20]:
report.correctness_by_question_type()

,correctness
question_type,
simple,1.0


We can also display the specific failures.

In [21]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness
id,,,,,,,


In [22]:
report.to_html("report.html")